In [5]:
import pandas as pd
import xml.etree.ElementTree as ET

# Handle the data from xml file to DataFrame that can be worked on
def parse_itunes_xml(xml_path):
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Locate the "Tracks" dictionary
    tracks_dict = root.find(".//dict/dict")
    
    # Extract track data
    tracks = []
    for track in tracks_dict.findall("dict"):
        track_data = {}
        key = None
        for element in track:
            # Detect <key> and save it for the next element
            if element.tag == "key":
                key = element.text
            elif key:
                # Handle different data types
                if element.tag == "integer":
                    track_data[key] = int(element.text)
                elif element.tag == "string":
                    track_data[key] = element.text
                elif element.tag == "date":
                    track_data[key] = element.text
                key = None  # Reset key after saving
        tracks.append(track_data)

    # Convert to DataFrame
    df = pd.DataFrame(tracks)
    return df

xml_path = r"C:\Users\dungt\OneDrive\Music.xml"
df = parse_itunes_xml(xml_path)

# Display the DataFrame
print(df.head())


   Track ID                                    Name                 Artist  \
0      1847                                      If                 ATVNCG   
1      1849                                     Bay                 ATVNCG   
2      1851  I Don't Do Drugs (feat. Ariana Grande)               Doja Cat   
3      1853                               You Right  Doja Cat & The Weeknd   
4      1855                           Love To Dream               Doja Cat   

                                             Album  \
0  Anh Trai Vượt Ngàn Chông Gai 2024 - Công Diễn 5   
1  Anh Trai Vượt Ngàn Chông Gai 2024 - Công Diễn 5   
2                              Planet Her (Deluxe)   
3                              Planet Her (Deluxe)   
4                              Planet Her (Deluxe)   

                         Kind     Size  Total Time    Year  \
0             MPEG audio file  7827346      244416  2024.0   
1             MPEG audio file  7031007      218784  2024.0   
2  Apple Music AAC a

In [7]:
# Total play counts
tot_play = df["Play Count"].sum()

# Total songs
tot_song = df["Name"].count()

# Total artists
tot_artist = df["Artist"].nunique()

# Total albums
tot_album = df["Album"].nunique()

# Make a summary table of total numbers
import hvplot.pandas
summary_df = pd.DataFrame({'Title':['Total Songs', 'Total Plays','Total Artists','Total Albums'],
                           'Value':[f'{tot_song} songs', f'{tot_play} plays', f'{tot_artist} artists', 
                                    f'{tot_album} albums']})

summary_table = summary_df.hvplot.table()
summary_table

:Table   [Title,Value]

In [12]:
# See the most played song in my playlist
most_played_song = df[df['Play Count'] == df['Play Count'].max()]

for _, row in most_played_song.iterrows():
    print(f'Song with the Most Play Count: {row["Name"]} by {row["Artist"]} with {row["Play Count"]} plays')

# Most played artist
artist_played = df.groupby('Artist')['Play Count'].sum()
most_played_artist = artist_played.idxmax()
max_artist_pcount = artist_played.max()

print(f"Artist with most plays: {most_played_artist} with {max_artist_pcount} plays")

# Most played album
album_played = df.groupby('Album')['Play Count'].sum()
most_played_album = album_played.idxmax()
max_album_pcount = album_played.max()

print(f'Album with most plays: {most_played_album} with {max_album_pcount} plays')



Song with the Most Play Count: Tình Yêu Vĩ Mô by Wren Evans & itsnk with 124.0 plays
Artist with most plays: Wren Evans & itsnk with 966.0 plays
Album with most plays: LOI CHOI: The Neo Pop Punk with 832.0 plays


In [13]:
# See the play counts of each songs in my playlist
play_per_song = df.groupby('Name', as_index=False)['Play Count'].sum()

# Convert it into a table
sort_play_per_song = play_per_song.sort_values(by='Play Count',ascending=[False])
play_per_song_table = sort_play_per_song.hvplot.table()
play_per_song_table

:Table   [Name,Play Count]

In [14]:
# See the play counts of each artist in my playlist
play_per_artist = df.groupby('Artist', as_index=False)['Play Count'].sum()

# Convert it into a table
sort_play_per_artist = play_per_artist.sort_values(by='Play Count',ascending=[False])
play_per_artist_table = sort_play_per_artist.hvplot.table()
play_per_artist_table

:Table   [Artist,Play Count]

In [15]:
# See the play counts of each album in my playlist
play_per_album = df.groupby('Album', as_index=False)['Play Count'].sum()

# Convert it into a table
sort_play_per_album = play_per_album.sort_values(by='Play Count',ascending=[False])
play_per_album_table = sort_play_per_album.hvplot.table()
play_per_album_table

:Table   [Album,Play Count]

In [16]:
# See the number of songs in each genre in my playlist
genre_counts = df["Genre"].value_counts()

print("Most Common Genres:")
print(genre_counts)

Most Common Genres:
Genre
Pop                    853
K-Pop                  766
R&B/Soul               420
Alternative            265
Hip-Hop/Rap            171
Classical               50
Indie Pop               47
Rock                    47
Electronic              45
Jazz                    32
J-Pop                   30
Rap                     21
Neo-Soul                19
Hip-Hop                 17
Dance                   13
Pop/Rock                12
Hard Rock               12
Soundtrack              12
Worldwide                8
Indie Rock               7
Comedy                   6
French Pop               6
Soul                     5
Classical Crossover      5
Singer/Songwriter        4
Disco                    4
Vocal Pop                3
Funk                     3
Christmas: Classic       3
Mandopop                 2
Korean Rock              2
Korean Hip-Hop           2
Country                  2
Pop Latino               2
Musicals                 2
Folk                     2
Vo

In [17]:
# Count the number of songs in each genre
genre_counts = df['Genre'].value_counts().head(20)

# Count the total songs:
tot_counts = len(df)
print("Genre:")
# Count the percentage of songs in each genre
for genre, count in genre_counts.items():
    genre_percentage = round(count / tot_counts *100,2)
    print(f'{genre} - {genre_percentage}%')

Genre:
Pop - 29.24%
K-Pop - 26.26%
R&B/Soul - 14.4%
Alternative - 9.08%
Hip-Hop/Rap - 5.86%
Classical - 1.71%
Indie Pop - 1.61%
Rock - 1.61%
Electronic - 1.54%
Jazz - 1.1%
J-Pop - 1.03%
Rap - 0.72%
Neo-Soul - 0.65%
Hip-Hop - 0.58%
Dance - 0.45%
Pop/Rock - 0.41%
Hard Rock - 0.41%
Soundtrack - 0.41%
Worldwide - 0.27%
Indie Rock - 0.24%


In [18]:
# See the number of songs by each artist in my playlist
artist_counts = df["Artist"].value_counts()

print("Most Common Artists:")
print(artist_counts)


Most Common Artists:
Artist
BTS                        134
Ariana Grande               73
Lana Del Rey                71
Epik High                   55
Taylor Swift                55
                          ... 
Julia Michaels               1
OoOo                         1
Lay Bankz                    1
Agust D & NiiHWA             1
Kay Trần & Bùi Công Nam      1
Name: count, Length: 713, dtype: int64


In [19]:
# Count the songs by each artist
artist_counts = df["Artist"].value_counts().head(10)

# Count the total songs
tot_counts = len(df)
print("Artist:")

# Count the percentage of songs by each artist
for artist, count in artist_counts.items():
    artist_percentage = round(count / tot_counts *100,2)
    print(f'{artist} - {artist_percentage}%')

Artist:
BTS - 4.59%
Ariana Grande - 2.5%
Lana Del Rey - 2.43%
Epik High - 1.89%
Taylor Swift - 1.89%
Doja Cat - 1.27%
Camila Cabello - 1.2%
IVE - 1.13%
LE SSERAFIM - 1.13%
IU - 1.13%


In [20]:
# See the number of tracks in each album in my playlist
album_counts = df["Album"].value_counts()

print("Albums with the most tracks:")
print(album_counts)


Albums with the most tracks:
Album
Born to Die – Paradise Edition (Special Version)    24
Complete Jazz Series 1940                           24
Querencia                                           21
Love. Angel. Music. Baby (Deluxe Version)           20
Baduizm                                             20
                                                    ..
Shine                                                1
Make You Mine - Single                               1
On My Mama - Single                                  1
Moment - Single                                      1
Hooked On a Feeling - Single                         1
Name: count, Length: 893, dtype: int64


In [21]:
import hvplot.pandas
# Make a table showing the genre coverage in my playlist
genre_counts = df['Genre'].value_counts().head(20)
tot_counts = len(df)

summary_df2 = pd.DataFrame(columns=['Genre','%'])
for genre, count in genre_counts.items():
    genre_percentage = round(count / tot_counts *100,2)
    summary_df2 = pd.concat([summary_df2,pd.DataFrame({'Genre':[genre],
                           '%':f'{genre_percentage}%'})])
    
genre_summary = summary_df2.hvplot.table()
genre_summary

:Table   [Genre,%]

In [22]:
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# Turn the genre coverage into an interactive bar chart
genre_counts = df["Genre"].value_counts().head(20)
tot_counts = len(df)
genres =[]
percentages=[]
for genre, count in genre_counts.items():
    genre_percentage = round(count / tot_counts *100,2)
    genres.append(genre)
    percentages.append(genre_percentage)

genre_df = pd.DataFrame({'Genre':genres, 'Percentage':percentages})
genre_chart = genre_df.hvplot.bar(x='Genre',y='Percentage',color='lightblue',title='Most common genres').opts(ylim=(0,30),shared_axes=False)
genre_chart

:Bars   [Genre]   (Percentage)

In [23]:
# Make a table showing the artist coverage in my playlist
artist_counts = df['Artist'].value_counts().head(20)
tot_counts = len(df)
summary_df3=pd.DataFrame(columns=['Artist','%'])

for artist, count in artist_counts.items():
    artist_percentage = round(count / tot_counts *100,2)
    summary_df3 = pd.concat([summary_df3,pd.DataFrame({'Artist':[artist],
                           '%':f'{artist_percentage}%'})])
    
artist_summary = summary_df3.hvplot.table(min_width={'Artist', '%'})
artist_summary

:Table   [Artist,%]

In [24]:
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# Turn the artist coverage into an interactive bar chart
artist_counts = df["Artist"].value_counts().head(20)
tot_counts = len(df)
artists =[]
percentages=[]
for artist, count in artist_counts.items():
    artist_percentage = round(count / tot_counts *100,2)
    artists.append(artist)
    percentages.append(artist_percentage)

artist_df = pd.DataFrame({'Artist':artists, 'Percentage':percentages})
artist_chart = artist_df.hvplot.bar(x='Artist',y='Percentage',color='green',title='Most common artists').opts(ylim=(0,5),shared_axes=False)
artist_chart

:Bars   [Artist]   (Percentage)

In [28]:
# Final dashboard
import panel as pn
pn.extension('tabulator')

# Showing all the tables and charts on a "My Playlist Summary" dashboard
template = pn.template.FastListTemplate(
    title = "My Playlist Summary",
    logo = r"C:\Users\dungt\Downloads\35c1c4bd969123cf7a80.jpg",
    header_background = "lightblue",
    main =[
        pn.Row(summary_table,play_per_song_table,play_per_artist_table,play_per_album_table,width=300,height=200),
        pn.GridBox(artist_summary,artist_chart,ncols=2,align='start',sizing_mode='stretch_width'),
        pn.GridBox(genre_summary,genre_chart,ncols=2,align='start',sizing_mode='stretch_width')
    ]
)
template.show()
        

Launching server at http://localhost:55047
